## Link to my drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1

Mounted at /gdrive
/gdrive/My Drive/[2023-2024] AN2DL/Homework 1


## Libraries, warnings, etc..

In [ ]:
seed = 69

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [ ]:
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns


### Images Export and Import before and after cleaning (not needed once you have dataset_wo_duplicates.npz)

In [ ]:
#Save the images in drive
num_images = 5200
for i in range(num_images):
  # Reorder the channels if necessary (for BGR to RGB conversion)
  image = cv2.cvtColor(dataset['data'][i], cv2.COLOR_RGB2BGR)

  #save imageXXXX.png
  cv2.imwrite("./dataset/image"+str(i).zfill(4)+".png",image)
  #cv2.imwrite("./dataset2/image"+str(i).zfill(4)+dataset['labels'][i]+".png",image)

  #check savings
  if i%100 == 0:
    print(str(i).zfill(4))


In [ ]:
from sys import breakpointhook
# Specify the folder path where your images are located
folder_path = './dataset_wo_duplicates'

# Use os.listdir to get a list of all files in the folder
files = os.listdir(folder_path)

# Filter the list to include only image files (e.g., .jpg, .png, .jpeg, etc.)
image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'))]

# Data array of numpy file
#data = np.ndarray(shape=(96,96,3),dtype='float32')
image_list = []

# Labels array of numpy file
labels = np.empty(0, dtype=object)

# Iterate through the list of image files and read each image using cv2
for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_file)

    # Retrieve id
    image_id =int(image_file[5:9])
    #print(image_id)

    # Use cv2.imread to read the image
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    #print(image)

    # Add to Data and Labels
    image_list.append(image)
    #data = np.append(data,image,axis=0)
    labels = np.append(labels,dataset['labels'][image_id])

# Convert the list of images to a NumPy array
data = np.array(image_list, dtype='float32')
#print(data)
np.savez('dataset_wo_duplicates.npz', data=data,labels=labels)

## Loading dataset:



In [ ]:
# unzip = False

# if unzip:
#    !unzip public_data.zip

dataset = np.load('dataset_wo_duplicates.npz', allow_pickle=True)

In [ ]:
def load_images():
  '''
  Function to load rescaled and correctly sized pictures into a np.array
  '''
    images = []
    for item in dataset['data']:
        img = item
        img = (img / 255).astype(np.float32)
        img = tfkl.Resizing(96,96)(img)
        if img is not None:
            images.append(img)
    return np.array(images)

In [ ]:
# Loading images, labels and using one-hot encoding.
X = load_images()
y = dataset['labels']
_ , y = np.unique(y, return_inverse=True)

from collections import Counter
counter = Counter(y)

y = tfk.utils.to_categorical(y, 2)

In [ ]:
# Showing some of the images to check that everything works fine
num_img = 15

fig, axes = plt.subplots(3, num_img//3, figsize=(15, 9))
for i in range(num_img):
    ax = axes[i%3, i%num_img//3]
    ax.imshow(np.clip(X[i], 0, 255))
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Splitting data
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=.1, stratify=np.argmax(y,axis=1))
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=len(X_test), stratify=np.argmax(y_train_val,axis=1))

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1]
batch_size = 64
epochs = 200
patience = 25
metadata = {}  # Dictionary for utility when plotting and compare how well (or not) models do

# Early stopping implementation
callbacks = [
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, restore_best_weights=True, mode='auto'),
]

print(f"Input Shape: {input_shape}, Output Shape: {output_shape}, Batch Size: {batch_size}, Epochs: {epochs}")

In [ ]:
def print_histories(metadata):
  '''
  Utility function that based on dictionary given as input, plots each model's history in terms of (val_)loss and (val_) accuracy
  '''
  plt.figure(figsize=(15,5))
  for k in list(metadata.keys()):
    plt.plot(metadata[k]['history']['loss'], alpha=.25, color=metadata[k]['color'][0], linestyle='--')
    plt.plot(metadata[k]['history']['val_loss'], label=k, alpha=.9, color=metadata[k]['color'][0])
  plt.legend(loc='upper left')
  plt.title('Categorical Crossentropy')
  plt.grid(alpha=.15)
  plt.ylim([0,1])
  plt.figure(figsize=(15,5))
  for k in list(metadata.keys()):
    be = metadata[k]['best_epoch']
    bescore = metadata[k]['history']['val_accuracy'][be]
    plt.plot(metadata[k]['history']['accuracy'], alpha=.25, color=metadata[k]['color'][1], linestyle='--')
    plt.plot(metadata[k]['history']['val_accuracy'], label=k, alpha=.9, color=metadata[k]['color'][1])
    plt.plot(be, bescore, marker='*', color=metadata[k]['color'][1], markersize=15)
    plt.text(0.95*be, 1.02*bescore,
             f'{bescore:.{3}g}', fontsize=12, color=metadata[k]['color'][1])
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=.15)

  plt.show()

In [ ]:
def print_matrixandstats(preds, y_test):
  '''
  Utility function that plots confmatrix and statistics useful to check model performances
  '''
  confmat = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))

  accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))
  precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
  recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
  f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')

  print('Accuracy:', accuracy.round(4))
  print('Precision:', precision.round(4))
  print('Recall:', recall.round(4))
  print('F1:', f1.round(4))

  plt.figure(figsize=(10, 8))
  sns.heatmap(confmat.T, xticklabels=list(('Unhealthy','Healthy')), yticklabels=list(('Unhealthy','Healthy')), cmap='Greens', annot=True)
  plt.xlabel('True labels')
  plt.ylabel('Predicted labels')
  plt.show()

## Adding some dropout

In [ ]:
def build_CNND(dropout_rate, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = tfkl.Conv2D(filters=32, kernel_size=4, padding='same', name='conv0')(input_layer)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=64, kernel_size=4, padding='same', name='conv1')(dropout)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=128, kernel_size=4, padding='same', name='conv2')(dropout)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=256, kernel_size=4, padding='same', name='conv3')(dropout)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=512, kernel_size=4, padding='same', name='conv4')(dropout)
    x = tfkl.ReLU(name='relu4')(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    output_layer = tfkl.Dense(units=output_shape, activation='softmax',name='Output')(dropout)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNND')

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

    return model

In [ ]:
# If more, doesn't learn a thing
dropout_rate = 1/8

CNND_model = build_CNND(dropout_rate, input_shape)
CNND_model.summary()
tfk.utils.plot_model(CNND_model, show_shapes=True, expand_nested=True)

In [ ]:
history = CNND_model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
# Metadata: made by the model and its history, chooses a color to be displayed and holds best epoch (just in case Early Stopping isn't used)
metadata['CNN w Dropout'] = {
    'model': CNND_model,
    'history': history,
    'color': ('#f52f07', '#f52f07'),
    'best_epoch': np.argmax(history['val_accuracy'])
}

In [ ]:
CNND_model.save('CNN w Dropout')

del CNND_model

CNND_model = tfk.models.load_model('CNN w Dropout')

In [ ]:
preds = CNND_model.predict(X_test, verbose=0)

print("Predictions Shape:", preds.shape)

print_matrixandstats(preds, y_test)

## Small modifies



In [ ]:
# Modifying learning rate, kernel_size, adding an intermediate Dense
def build_CNNDv2(dropout_rate, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='conv0')(input_layer)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='conv1')(dropout)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=128, kernel_size=3, padding='same', name='conv2')(dropout)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=256, kernel_size=3, padding='same', name='conv3')(dropout)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=512, kernel_size=3, padding='same', name='conv4')(dropout)
    x = tfkl.ReLU(name='relu4')(x)
    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Dense(units=256, activation='relu')(dropout)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    output_layer = tfkl.Dense(units=output_shape, activation='softmax',name='Output')(dropout)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNNDv2')

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])

    return model

In [ ]:
CNNDv2_model = build_CNNDv2(dropout_rate, input_shape)
CNNDv2_model.summary()
tfk.utils.plot_model(CNNDv2_model, show_shapes=True, expand_nested=True)

In [ ]:
history = CNNDv2_model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
metadata['CNN w Dropout v2'] = {
    'model': CNNDv2_model,
    'history': history,
    'color': ('#0990de', '#0990de'),
    'best_epoch': np.argmax(history['val_accuracy'])
}

In [ ]:
CNNDv2_model.save('CNN w Dropout v2')

del CNNDv2_model

CNNDv2_model = tfk.models.load_model('CNN w Dropout v2')

preds = CNNDv2_model.predict(X_test, verbose=0)

print("Predictions Shape:", preds.shape)

In [ ]:
confmat = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))

accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(preds, axis=-1), average='macro')

print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

plt.figure(figsize=(10, 8))
sns.heatmap(confmat.T, xticklabels=list(('Unhealthy','Healthy')), yticklabels=list(('Unhealthy','Healthy')), cmap='Greens', annot=True)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

## Adding regularizers

In [ ]:
def build_CNNDv3(dropout_rate, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)
    l_lambda = 0.0075

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='conv0')(input_layer)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='conv1')(dropout)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=128, kernel_size=3, padding='same', name='conv2')(dropout)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=256, kernel_size=3, padding='same', name='conv3')(dropout)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=512, kernel_size=3, padding='same', name='conv4')(dropout)
    x = tfkl.ReLU(name='relu4')(x)
    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l_lambda))(dropout)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    output_layer = tfkl.Dense(units=output_shape, activation='softmax',name='Output', kernel_regularizer=tf.keras.regularizers.L1L2(l_lambda))(dropout)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNNDv3')

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])

    return model

In [ ]:
CNNDv3_model = build_CNNDv3(dropout_rate, input_shape)
CNNDv3_model.summary()
tfk.utils.plot_model(CNNDv3_model, show_shapes=True, expand_nested=True)

In [ ]:
history = CNNDv3_model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
metadata['CNN w Dropout v3'] = {
    'model': CNNDv3_model,
    'history': history,
    'color': ('#7f0bde', '#7f0bde'),
    'best_epoch': np.argmax(history['val_accuracy'])
}

In [ ]:
CNNDv3_model.save('CNN w Dropout v3')

del CNNDv3_model

In [ ]:
CNNDv3_model = tfk.models.load_model('CNN w Dropout v3')

In [ ]:
preds = CNNDv3_model.predict(X_test, verbose=0)
print("Predictions Shape:", preds.shape)

print_matrixandstats(preds, y_test)

## Adding some augmentation

In [ ]:
def build_CNNDA(dropout_rate, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)
    l_lambda = 0.0005

    preprocessing = tf.keras.Sequential([tfkl.RandomRotation(0.3),
                                         tfkl.RandomTranslation(0.25,0.25),
                                         tfkl.RandomFlip("horizontal")], name='preprocessing')

    input_layer = tfkl.Input(shape=input_shape, name='Input')
    preprocessing = preprocessing(input_layer)

    x = tfkl.Conv2D(filters=32, kernel_size=3, padding='same', name='conv0')(preprocessing)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='conv1')(dropout)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=128, kernel_size=3, padding='same', name='conv2')(dropout)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=256, kernel_size=3, padding='same', name='conv3')(dropout)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=512, kernel_size=3, padding='same', name='conv4')(dropout)
    x = tfkl.ReLU(name='relu4')(x)
    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l_lambda))(dropout)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    output_layer = tfkl.Dense(units=output_shape, activation='softmax',name='Output', kernel_regularizer=tf.keras.regularizers.L1L2(l_lambda))(dropout)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNNDA')

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])

    return model

In [ ]:
CNNDA_model = build_CNNDA(dropout_rate, input_shape)
CNNDA_model.summary()
tfk.utils.plot_model(CNNDA_model, show_shapes=True, expand_nested=True)

In [ ]:
history = CNNDA_model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
metadata['CNND Augmentation'] = {
    'model': CNNDA_model,
    'history': history,
    'color': ('#0bd98a', '#0bd98a'),
    'best_epoch': np.argmax(history['val_accuracy'])
}

In [ ]:
CNNDA_model.save('CNND Augmentation')
#CNNDA_model.save('CNND Augmentationx3')

del CNNDA_model

In [ ]:
CNNDA_model = tfk.models.load_model('CNND Augmentation')
#CNNDA_model = tfk.models.load_model('CNND Augmentationx3')

In [ ]:
preds = CNNDA_model.predict(X_test, verbose=0)
print("Predictions Shape:", preds.shape)

print_matrixandstats(preds, y_test)

## Basic Model with added augmentation

In [ ]:
# Taking back the kernel size to 4
def build_CNND_BaseA(dropout_rate, input_shape=input_shape, output_shape=output_shape):
    tf.random.set_seed(seed)

    preprocessing = tf.keras.Sequential([tfkl.RandomRotation(0.3),
                                         tfkl.RandomTranslation(0.25,0.25),
                                         tfkl.RandomFlip("horizontal")], name='preprocessing')

    input_layer = tfkl.Input(shape=input_shape, name='Input')
    preprocessing = preprocessing(input_layer)

    x = tfkl.Conv2D(filters=32, kernel_size=4, padding='same', name='conv0')(preprocessing)
    x = tfkl.ReLU(name='relu0')(x)
    x = tfkl.MaxPooling2D(name='mp0')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=64, kernel_size=4, padding='same', name='conv1')(dropout)
    x = tfkl.ReLU(name='relu1')(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=128, kernel_size=4, padding='same', name='conv2')(dropout)
    x = tfkl.ReLU(name='relu2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=256, kernel_size=4, padding='same', name='conv3')(dropout)
    x = tfkl.ReLU(name='relu3')(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    x = tfkl.Conv2D(filters=512, kernel_size=4, padding='same', name='conv4')(dropout)
    x = tfkl.ReLU(name='relu4')(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)
    dropout = tfkl.Dropout(dropout_rate, seed=seed)(x)

    output_layer = tfkl.Dense(units=output_shape, activation='softmax',name='Output')(dropout)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CNND_BaseA')

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

    return model

In [ ]:
CNND_BaseAmodel = build_CNND_BaseA(dropout_rate, input_shape)
CNND_BaseAmodel.summary()
tfk.utils.plot_model(CNND_BaseAmodel, show_shapes=True, expand_nested=True)

In [ ]:
history = CNND_BaseAmodel.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = callbacks
).history

In [ ]:
metadata['CNN w Dropout Base+A'] = {
    'model': CNND_BaseAmodel,
    'history': history,
    'color': ('#db18d8', '#db18d8'),
    'best_epoch': np.argmax(history['val_accuracy'])
}

In [ ]:
CNND_BaseAmodel.save('CNN w Dropout Base+A')

del CNND_BaseAmodel

In [ ]:
CNND_BaseAmodel = tfk.models.load_model('CNN w Dropout Base+A')

In [ ]:
preds = CNND_BaseAmodel.predict(X_test, verbose=0)

print("Predictions Shape:", preds.shape)

In [ ]:
print_matrixandstats(preds, y_test)

## Models comparison

In [ ]:
print_histories(metadata)